In [1]:
import json
import numpy as np
import pandas as pd

from scipy.misc import imresize
from scipy.ndimage.interpolation import rotate

# Keras imports
from keras.models import Sequential, Model
from keras.layers import Dense, Add, Activation, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, Input, LeakyReLU, BatchNormalization, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg19 import VGG19
from keras.optimizers import RMSprop, Adagrad, SGD
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


### Pretrained model

In [2]:
base_model = VGG19(weights='imagenet')
vgg19 = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

### Load training and test data

In [3]:
train_dir = '/Users/dcromp/Documents/projects/ice_bergs/data/processed/train.json'
test_dir = '/Users/dcromp/Documents/projects/ice_bergs/test/processed/test.json'
batch_size = 32

# Load data and calculate batch sizes
with open(train_dir) as data_file:    
    train_json = json.load(data_file)
    whole_batch = int(len(train_json) / batch_size)
    over_batch = int(len(train_json) % batch_size)
    total_batches = whole_batch
    if over_batch != 0:
        total_batches = total_batches + 1    
        
with open(test_dir) as data_file:    
    test_json = json.load(data_file)

In [4]:
def data_avg(train_json, test_json):
    
    # Calculate size of data set
    num_samples = len(train_json) + len(test_json)
    channel_dim = len(train_json[0]['band_1'])
    channels = 2
    
    #Initlize sum and counts
    band_1_sum = 0
    band_2_sum = 0
    angle_sum = 0
    angle_count = 0
    band_1_max = 0
    band_1_min = 0
    band_2_max = 0
    band_2_min = 0
    
    # Loop over training data
    for sample in train_json:
        band_1_sum = band_1_sum + sum(sample['band_1'])
        band_2_sum = band_2_sum + sum(sample['band_2'])
        
        if max(sample['band_1']) > band_1_max:
            band_1_max = max(sample['band_1'])
        if min(sample['band_1']) < band_1_min:
            band_1_min = min(sample['band_1'])
            
        if max(sample['band_2']) > band_2_max:
            band_2_max = max(sample['band_2'])
        if min(sample['band_2']) < band_2_min:
            band_2_min = min(sample['band_2'])
        
        if isinstance(sample['inc_angle'], str):
            pass
        else:
            angle_sum = angle_sum + sample['inc_angle']
            angle_count = angle_count + 1
            
    # Loop over test data
    for sample in test_json:
        band_1_sum = band_1_sum + sum(sample['band_1'])
        band_2_sum = band_2_sum + sum(sample['band_2'])
        
        
        if max(sample['band_1']) > band_1_max:
            band_1_max = max(sample['band_1'])
        if min(sample['band_1']) < band_1_min:
            band_1_min = min(sample['band_1'])
            
        if max(sample['band_2']) > band_2_max:
            band_2_max = max(sample['band_2'])
        if min(sample['band_2']) < band_2_min:
            band_2_min = min(sample['band_2'])
        
        if isinstance(sample['inc_angle'], str):
            pass
        else:
            angle_sum = angle_sum + sample['inc_angle']
            angle_count = angle_count + 1
    
    # Calculate averages
    band_1_avg = band_1_sum / (num_samples * channel_dim)
    band_2_avg = band_2_sum / (num_samples * channel_dim)
    angle_avg = angle_sum / angle_count
    
    return band_1_avg, band_2_avg, angle_avg, band_1_max, band_1_min, band_2_max, band_2_min

In [5]:
band_1_avg, band_2_avg, angle_avg, band_1_max, band_1_min, band_2_max, band_2_min = data_avg(train_json, test_json)

In [6]:
# Extract labels
labels = np.zeros(len(train_json))
for index, sample in enumerate(train_json):
    labels[index] = sample['is_iceberg']
    
# Extract IDs
test_id = []
for index, sample in enumerate(test_json):
    test_id.append(sample['id'])

In [7]:
# Extract samples
def sample_extractor(data_json):
    
    samples = np.zeros((len(data_json), 75, 75, 2))
    angles = np.zeros(len(data_json))

    for index, sample in enumerate(data_json):
        band_1 = np.asarray(sample['band_1']).reshape(75,75)
        band_2 = np.asarray(sample['band_2']).reshape(75,75)
        samples[index][:,:,0] = band_1
        samples[index][:,:,1] = band_2
        inc_angle = sample['inc_angle']
        if isinstance(sample['inc_angle'], str):
            angles[index] = angle_avg
        else: 
            angles[index] = sample['inc_angle']
            
    return samples, angles
    
train_samples, train_angles = sample_extractor(train_json)
test_samples, test_angles = sample_extractor(test_json)

In [8]:
# Minus the mean from the samples
def mean_zero(samples, angles, band_1_avg, band_2_avg, angle_avg):
    for index, sample in enumerate(samples):
        samples[index][:,:,0] = np.subtract(sample[:,:,0], band_1_avg)
        samples[index][:,:,1] = np.subtract(sample[:,:,1], band_2_avg)
        angles[index] = np.subtract(angles[index], angle_avg)
    return samples, angles

train_samples, train_angles = mean_zero(train_samples, train_angles, band_1_avg, band_2_avg, angle_avg)
test_samples, test_angles = mean_zero(test_samples, test_angles, band_1_avg, band_2_avg, angle_avg)
print(train_samples.shape)


def scale(train, test):
    print(train.shape)
    samples = np.concatenate((train, test), axis=0)

    # Scale the samples between 0-1
    samples[:,:,:,0] = samples[:,:,:,0] + abs(np.amin(samples[:,:,:,0]))
    samples[:,:,:,1] = samples[:,:,:,1] + abs(np.amin(samples[:,:,:,1]))
        
    samples[:,:,:,0] = samples[:,:,:,0] / (np.amax(samples[:,:,:,0]))
    samples[:,:,:,1] = samples[:,:,:,1] / (np.amax(samples[:,:,:,1]))
    
    print(np.amax(samples[:,:,:,0]))
    print(np.amax(samples[:,:,:,1]))
        
    print(np.amin(samples[:,:,:,0]))
    print(np.amin(samples[:,:,:,1]))
        
    return samples[:train.shape[0]], samples[train.shape[0]:]

train_samples, test_samples = scale(train_samples, test_samples)
print(train_samples.shape)

"""def scale_angles(train, test):
    print(train.shape)
    samples = np.concatenate((train, test), axis=0)

    # Scale the samples between 0-1
    samples = samples + abs(np.amin(samples))   
    samples = samples / (np.amax(samples))
    
    print(np.amax(samples))
    print(np.amin(samples))
        
    return samples[:train.shape[0]], samples[train.shape[0]:]

train_angles, test_angles = scale_angles(train_angles, test_angles)
print(train_angles.shape)"""

(1604, 75, 75, 2)
(1604, 75, 75, 2)
1.0
1.0
0.0
0.0
(1604, 75, 75, 2)


'def scale_angles(train, test):\n    print(train.shape)\n    samples = np.concatenate((train, test), axis=0)\n\n    # Scale the samples between 0-1\n    samples = samples + abs(np.amin(samples))   \n    samples = samples / (np.amax(samples))\n    \n    print(np.amax(samples))\n    print(np.amin(samples))\n        \n    return samples[:train.shape[0]], samples[train.shape[0]:]\n\ntrain_angles, test_angles = scale_angles(train_angles, test_angles)\nprint(train_angles.shape)'

In [9]:
# Layer definitions for more complex networks

def conv_block(x, filters, size, stride=(2,2), mode='same', act=True):
    x = Conv2D(filters, size, size, subsample=stride, border_mode=mode)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x) if act else x

def res_block(ip, nf=15):
    shortcut = ip
    x = conv_block(ip, nf, 3, (1,1))
    x = conv_block(x, nf, 3, (1,1), act=False)
    # Check if we need to increase dimensionality of the shortcut
    if shortcut.shape[3] != x.shape[3]:
        shortcut = conv_block(shortcut, nf, 1, (1,1), act=False)
    return Add()([x, shortcut])

def deconv_block(x, filters, size, shape, stride=(2,2)):
    x = Deconvolution2D(filters, size, size, subsample=stride, 
        border_mode='same', output_shape=(None,)+shape)(x)
    x = BatchNormalization(mode=2)(x)
    return Activation('relu')(x)

def up_block(x, filters, size):
    x = keras.layers.UpSampling2D()(x)
    x = Convolution2D(filters, size, size, border_mode='same')(x)
    x = BatchNormalization(mode=2)(x)
    return Activation('relu')(x)

In [10]:
# Resnet model

# Inputs
band_input = Input(shape=(75, 75, 2), name='band_input')
angle_input = Input(shape=(1,), name='angle_input')

x=conv_block(band_input, 15, 3, (1,1))

# 13 layer resnet
for i in range(3): x=res_block(x)
for i in range(4): x=res_block(x, 30)
for i in range(6): x=res_block(x, 60)
    
x = AveragePooling2D(pool_size=2)(x)
    
cnn_out = Flatten()(x)

x = Dropout(0.2)(cnn_out)
x = Dense(1000, activation='relu')(cnn_out)
x = Dropout(0.2)(x)
x = concatenate([x, angle_input])
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[band_input, angle_input], outputs=output)

model.summary()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/kernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, (3, 3), strides=(1, 1), padding="same")`
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/kernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, (3, 3), strides=(1, 1), padding="same")`
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/kernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, (1, 1), strides=(1, 1), padding="same")`
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/kernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(60, (3, 3), strides=(1, 1), padding="same")`
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/kernel/__main__.py:4: UserWarning:

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
band_input (InputLayer)         (None, 75, 75, 2)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 75, 75, 15)   285         band_input[0][0]                 
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 75, 75, 15)   60          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 75, 75, 15)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
model.compile(optimizer=RMSprop(lr=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Split out test data
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    assert len(b) == len(c)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

train_samples, train_angles, labels = unison_shuffled_copies(train_samples, train_angles, labels)
num_train = int(len(train_samples) * 0.2)

x_train = train_samples[num_train:]
x_2_train = train_angles[num_train:]
y_train = labels[num_train:]
x_test = train_samples[:num_train]
x_2_test = train_angles[:num_train]
y_test = labels[:num_train]

"""
# Create rotations of images
rotations = np.zeros((len(x_train), 75, 75, 2))
for index, sample in enumerate(x_train):
    rotation = rotate(sample, angle = np.random.randint(1, 10), reshape = False)
    rotations[index] = rotation
    
flips = np.zeros((len(x_train), 75, 75, 2))
for index, sample in enumerate(x_train):
    flip = np.fliplr(sample)
    flips[index] = flip
    
x_train = np.concatenate((x_train, rotations, flips), axis=0)
x_2_train = np.concatenate((x_2_train, x_2_train, x_2_train), axis=0)
y_train = np.concatenate((y_train, y_train, y_train), axis=0)


print(rotations.shape, x_train.shape)"""

'\n# Create rotations of images\nrotations = np.zeros((len(x_train), 75, 75, 2))\nfor index, sample in enumerate(x_train):\n    rotation = rotate(sample, angle = np.random.randint(1, 10), reshape = False)\n    rotations[index] = rotation\n    \nflips = np.zeros((len(x_train), 75, 75, 2))\nfor index, sample in enumerate(x_train):\n    flip = np.fliplr(sample)\n    flips[index] = flip\n    \nx_train = np.concatenate((x_train, rotations, flips), axis=0)\nx_2_train = np.concatenate((x_2_train, x_2_train, x_2_train), axis=0)\ny_train = np.concatenate((y_train, y_train, y_train), axis=0)\n\n\nprint(rotations.shape, x_train.shape)'

In [ ]:
# train the model
checkpointer = ModelCheckpoint(filepath='nov.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10)

model.fit([x_train, x_2_train], y_train, batch_size=32, shuffle=True, validation_split=0.2, callbacks=[checkpointer, earlystop], initial_epoch=0, epochs=2000)

Train on 1027 samples, validate on 257 samples
Epoch 1/2000
1027/1027 [==============================] - 629s 613ms/step - loss: 6.3976 - acc: 0.5862 - val_loss: 7.4005 - val_acc: 0.5409
Epoch 2/2000
1027/1027 [==============================] - 582s 566ms/step - loss: 6.7180 - acc: 0.5794 - val_loss: 7.4005 - val_acc: 0.5409
Epoch 3/2000
1027/1027 [==============================] - 558s 543ms/step - loss: 6.6870 - acc: 0.5813 - val_loss: 7.4605 - val_acc: 0.5370
Epoch 4/2000
 256/1027 [======>.......................] - ETA: 6:15 - loss: 6.9304 - acc: 0.5664

In [ ]:
model.load_weights('nov.weights.best.hdf5')

In [ ]:
# Resnet 13 layers
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Non Scale angle
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Scale angle
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# large first feature map
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# with flips, less rotation
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# with rotations
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# test original model
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Predictions
prediction_array = model.predict([test_samples, test_angles])

submission = pd.DataFrame()
submission['id'] = test_id
submission['is_iceberg']=prediction_array.reshape((prediction_array.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
def sample_extractor_3channel(data_json):
    
    samples = np.zeros((len(data_json), 224, 224, 3))
    angles = np.zeros(len(data_json))

    for index, sample in enumerate(data_json):
        band_1 = np.asarray(sample['band_1']).reshape(75,75)
        band_1 = imresize(band_1, (224,224))
        band_2 = np.asarray(sample['band_2']).reshape(75,75)
        band_2 = imresize(band_2, (224,224))
        band_3 = (band_1 + band_2) / 2
        samples[index][:,:,0] = band_1
        samples[index][:,:,1] = band_2
        samples[index][:,:,2] = band_3
        inc_angle = sample['inc_angle']
        if isinstance(sample['inc_angle'], str):
            angles[index] = angle_avg
        else: 
            angles[index] = sample['inc_angle']
            
    return samples, angles
    
train_samples, train_angles = sample_extractor_3channel(train_json)
test_samples, test_angles = sample_extractor_3channel(test_json)

def mean_zero(samples, angles, band_1_avg, band_2_avg, angle_avg):
    for index, sample in enumerate(samples):
        samples[index][:,:,0] = np.subtract(sample[:,:,0], band_1_avg)
        samples[index][:,:,1] = np.subtract(sample[:,:,1], band_2_avg)
        angles[index] = np.subtract(angles[index], angle_avg)
        
    
    # Scale the samples between 0-1
    samples[:,:,:,0] = samples[:,:,:,0] + abs(np.amin(samples[:,:,:,0]))
    samples[:,:,:,1] = samples[:,:,:,1] + abs(np.amin(samples[:,:,:,1]))
    samples[:,:,:,2] = samples[:,:,:,2] + abs(np.amin(samples[:,:,:,2]))
        
    samples[:,:,:,0] = samples[:,:,:,0] / (np.amax(samples[:,:,:,0]))
    samples[:,:,:,1] = samples[:,:,:,1] / (np.amax(samples[:,:,:,1]))
    samples[:,:,:,2] = samples[:,:,:,2] / (np.amax(samples[:,:,:,2]))
    
    print(np.amax(samples[:,:,:,0]))
    print(np.amax(samples[:,:,:,1]))
    print(np.amax(samples[:,:,:,2]))
        
    print(np.amin(samples[:,:,:,0]))
    print(np.amin(samples[:,:,:,1]))
    print(np.amin(samples[:,:,:,2]))
        
    return samples, angles

train_samples, train_angles = mean_zero(train_samples, train_angles, band_1_avg, band_2_avg, angle_avg)
test_samples, test_angles = mean_zero(test_samples, test_angles, band_1_avg, band_2_avg, angle_avg)

vgg19 = Model(inputs=base_model.input, outputs=base_model.get_layer('block2_pool').output)

bottleneck_samples = vgg19.predict(train_samples)



num_train = int(len(bottleneck_samples) * 0.2)

x_train = bottleneck_samples[num_train:]
y_train = labels[num_train:]

x_test = bottleneck_samples[:num_train]
y_test = labels[:num_train]

print(bottleneck_samples.shape)

In [ ]:
model_input = Input(shape=(56, 56, 128))
#x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(model_input)
#x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=4)(model_input)
cnn_out = Flatten()(x)

x = Dropout(0.2)(cnn_out)
x = Dense(500, activation='relu')(cnn_out)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model_input, outputs=output)

model.summary()

In [ ]:
rmsprop = RMSprop(lr=1)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train the model
checkpointer = ModelCheckpoint(filepath='nov.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10)

model.fit(x_train, y_train, batch_size=16, validation_split=0.2, callbacks=[checkpointer, earlystop], initial_epoch=0, epochs=200)

In [ ]:
# Functional model

# Inputs
band_input = Input(shape=(75, 75, 2), name='band_input')
angle_input = Input(shape=(1,), name='angle_input')


# CNN layers
#x = Conv2D(filters=64, kernel_size=9, strides=(2, 2), padding='same', activation='relu')(band_input)# Large feature map to start
x = Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(band_input)
x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = AveragePooling2D(pool_size=2)(x)
cnn_out = Flatten()(x)

x = Dropout(0.2)(cnn_out)
x = Dense(500, activation='relu')(cnn_out)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)
x = concatenate([x, angle_input])
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[band_input, angle_input], outputs=output)

model.summary()

In [ ]:
vgg19.summary()

In [ ]:
# Predictions
prediction_array = model.predict([test_samples, test_angles])

In [ ]:
# vgg 19 inspired
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# avg pooling
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# added deep layer
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# new functions
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# dropout 0.2
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# earlystop
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Increase dropout to 0.4 for dense layers
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# angle last year, less deep layers
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
#moved angle one layer before the end
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
#No batch norm
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# More deep layers at end
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Angle added after first dense layer
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Dropout in cnn layers
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# Angle added to cnn out
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# angle as last layer
model.evaluate([x_test, x_2_test], y_test)

In [ ]:
# VGG 19 inspired

# Inputs
band_input = Input(shape=(75, 75, 2), name='band_input')
angle_input = Input(shape=(1,), name='angle_input')


# CNN layers
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(band_input)
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(band_input)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.2)(x)

x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.2)(x)

x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.2)(x)

x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.2)(x)

cnn_out = Flatten()(x)

x = Dense(2048, activation='relu')(cnn_out)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.2)(x)
x = concatenate([x, angle_input])
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[band_input, angle_input], outputs=output)

model.summary()

In [ ]:
# Functional model

# Inputs
band_input = Input(shape=(75, 75, 2), name='band_input')
angle_input = Input(shape=(1,), name='angle_input')


# CNN layers
x = Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(band_input)
x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = AveragePooling2D(pool_size=2)(x)
cnn_out = Flatten()(x)

x = Dropout(0.2)(cnn_out)
x = Dense(500, activation='relu')(cnn_out)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)
x = concatenate([x, angle_input])
x = Dropout(0.2)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[band_input, angle_input], outputs=output)

model.summary()

In [ ]:
"""
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', 
                        input_shape=(75, 75, 2))) 
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()
"""

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
samples.shape

In [ ]:
average = 0
for index, sample in enumerate(samples):
    average = average + np.average(samples[index][0])
average

In [ ]:
# Data is not normalised
sum(train_json[0]['band_1']) / len(train_json[0]['band_1'])

In [ ]:
np.asarray()

In [ ]:
75*75

In [ ]:
array = np.asarray([1,2,3,4]).reshape(2,2)

In [ ]:
np.add(array, 5)

In [ ]:
whole_batch = int(len(train_json) / batch_size)
over_batch = int(len(train_json) % batch_size)
print("{} data points, {} batches of size {}, {} data points left over".format(len(train_json), whole_batch, batch_size, over_batch))

In [ ]:
train_json[0]['band_1']